In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import json
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

openai_api_key = os.environ.get("OPENAI_API_KEY")

In [11]:
headers = {
    'Content-Type': 'application/json'
}
freshdesk_api_key = os.environ.get('FRESHDESK_API_KEY')
auth = (freshdesk_api_key, 'X')
domain = 'nueca'

In [9]:
customer_problem = "The customer inquired about the status of their order and whether it would be delivered after experiencing a delay. They expressed concern about the delivery timeline and requested confirmation on the delivery of their order."
agent_resolution = "The agent confirmed that delivery operations had resumed and that the customer's order would be delivered that afternoon. The agent provided an update on the delivery status and assured the customer that their order was on its way. [Status: RESOLVED]"

In [10]:
print(f"Customer problem: {customer_problem}\n\nAgent resolution: {agent_resolution}")

Customer problem: The customer inquired about the status of their order and whether it would be delivered after experiencing a delay. They expressed concern about the delivery timeline and requested confirmation on the delivery of their order.

Agent resolution: The agent confirmed that delivery operations had resumed and that the customer's order would be delivered that afternoon. The agent provided an update on the delivery status and assured the customer that their order was on its way. [Status: RESOLVED]


In [ ]:
prompt_template = """
## ROLE
You are analyzing customer support conversations for Tindahang Tapat, a digital platform enabling sari-sari stores to order groceries via mobile phone. Your task is to extract standardized problem-resolution pairs for a RAG knowledge base.

The CONVERSATION DATA contains messages from the customer and the agent (customer service representative).

## CONVERSATION DATA
{conversation}

## QUERY TYPE - This the general classification of the customer's concern, identified by the agent.
{query_type}

## SPECIFIC ISSUE - This is the specific classification of the customer's concern, identified by the agent.
{specific_issue}

## TASK
Extract the CORE PROBLEM STATEMENT and RESOLUTION based on the given CONVERSATION DATA, QUERY TYPE, and SPECIFIC ISSUE.

## OUTPUT REQUIREMENTS

### CORE PROBLEM STATEMENT
- Must be based on the main concern of the customer
- If multiple concerns exist, focus on the main one. ONLY ONE customer problem should be included in the CORE PROBLEM STATEMENT.
- Do NOT include assumptions about possible impact to the customer
- Focus only on problems explicitly stated by the customer
- If no problem exists, output: "No identifiable problem"
- Length: Exactly 2-3 sentences

### RESOLUTION
- Must be based on concrete actions taken by the agent to resolve the customer's main issue
- Promises, reassurances, explanations, or status updates do NOT count as resolution
- If no concrete resolution exists, output: "Unresolved"
- Length: Exactly 2-3 sentences for resolved cases, single word "Unresolved" for unresolved cases

## WHAT CONSTITUTES CONCRETE RESOLUTION
**RESOLVED status requires concrete action taken by the agent:**
- Refund processed and confirmed
- Replacement item dispatched
- Technical issue fixed (account unlocked, payment processed, etc.)
- Correct information provided with verification
- Problem directly addressed with measurable outcome
- Compensation or credits applied to account

## WHAT DOES NOT CONSTITUTE RESOLUTION
**The following are NOT resolutions and should be marked "Unresolved":**
- Promises or reassurances ("we will try", "team will attempt", "we will follow up")
- Explanations without solutions (explaining why problem occurred)
- Information gathering only (asking for details without solving)
- Escalations without final resolution
- Apologies without corrective action
- Status updates without problem resolution
- Acknowledgments of concerns without action

## OBJECTIVITY REQUIREMENTS
**ELIMINATE ALL EMOTIONAL AND SUBJECTIVE LANGUAGE:**

### Prohibited Emotional Language
- Customer emotions: "frustrated", "angry", "upset", "disappointed", "happy", "satisfied", "annoyed", "worried", "concerned", "stressed"
- Agent emotions: "apologetic", "sympathetic", "understanding", "helpful", "caring"
- Emotional reactions: "complained", "expressed dissatisfaction", "was pleased", "felt relieved"
- Subjective descriptions: "terrible experience", "great service", "poor quality", "excellent response"

### Required Objective Language
- Use factual, neutral verbs: "reported", "stated", "indicated", "requested", "received", "experienced"
- Focus on actions and events, not feelings about them
- Describe what happened, not how anyone felt about it
- Use technical, operational language

## NORMALIZATION RULES
Replace all specific details with standardized terms:

### Customer References
- Any customer name → "customer"
- Any store name → "customer's store"
- Any order number/ID → "customer order"
- Any account details → "customer account"
- Specific amounts → "order amount" or "payment amount"

### CRITICAL: Product Normalization
**ALWAYS replace ALL specific product names with "items"**
- "Nescafe 3-in-1 sachets" → "items"
- "Tide powder 1kg" → "items"
- "Lucky Me noodles" → "items"
- "Coca-Cola 1.5L" → "items"
- "Pantene shampoo" → "items"
- "Kopiko coffee" → "items"
- "Maggi seasoning" → "items"
- ANY brand name or specific product → "items"

### Time References
- Specific dates → "scheduled delivery date"
- Specific times → "delivery time"
- Time periods → "delivery window"

### Location References
- Specific addresses → "delivery location"
- Cities/regions → "delivery area"
- Landmarks → "delivery location"

### External Factors
- Weather events (storms, typhoons, floods) → "weather conditions"
- Traffic issues → "traffic conditions"
- System problems → "technical issues"

### Technical Terms
- "App crashed/froze/stopped/won't work" → "mobile application became unresponsive"
- "Payment failed/declined/error/won't process" → "payment processing failed"
- "Can't login/access/sign in" → "authentication failed"
- "Won't load/loading/stuck loading" → "content failed to load"

## CRITICAL INSTRUCTIONS
1. Read the conversation completely to identify the main customer concern
2. Apply ALL normalization rules - especially product normalization to "items"
3. Mark as "RESOLVED" ONLY if concrete action was taken to fix the problem
4. If only promises, explanations, or reassurances were given, mark as "Unresolved"
5. Remove all specific dates, times, locations, and external factors
6. Focus on the functional problem, not emotional context
7. Ensure language is generic and searchable for RAG retrieval

## OUTPUT FORMAT
Always return output in exactly the following format with no additional commentary, explanations, or formatting:

CORE PROBLEM STATEMENT:
[2-3 sentences describing the main customer concern, normalized]

RESOLUTION:
[2-3 sentences describing concrete actions taken by agent, normalized] [Status: RESOLVED] OR [Unresolved]
"""

In [33]:
prompt = prompt_template.format(conversation="", query_type="", specific_issue="")
# print(prompt)
a = encoding.encode(prompt)
len(a)

445

In [34]:
from qdrant_client import QdrantClient

qd_client = QdrantClient("localhost:8333")

In [ ]:
qd_client.create_payload_index()

In [45]:
import re

def split_problem_and_resolution(text) -> tuple[str, str]:
    pattern = r"""
        [#\s]*CORE\s+PROBLEM\s+STATEMENT[:\s]*
        (.*?)                                  
        [#\s]*RESOLUTION[:\s]*
        (.*?)                      
        [#\*\s]*OPERATION\s+TYPE[:\s\#\*]*
        (.*)                          
    """

    # pattern = r"""
    #     CORE\s+PROBLEM\s+STATEMENT:\s*
    #     (.*?)                                  
    #     \s*RESOLUTION:\s*
    #     (.*?)                      
    #     \s*OPERATION\s+TYPE:\s*
    #     (.*)                          
    # """

    match = re.search(pattern, text, re.DOTALL | re.VERBOSE)
    if match:
        problem = match.group(1).strip()
        resolution = match.group(2).strip()
        operation = match.group(3).strip()
        return problem, resolution, operation
    else:
        return None, None, None

a = "CORE PROBLEM STATEMENT: Customer reported receiving damaged items during delivery. Customer requested replacement or resolution for the damaged items.\n\nRESOLUTION: [Status: RESOLVED] Customer's issue with damaged items was reported to the warehouse for follow-up. The damaged item was to be adjusted in the customer's next delivery.\n\nOPERATION TYPE: typhoon"
split_problem_and_resolution(a)

('Customer reported receiving damaged items during delivery. Customer requested replacement or resolution for the damaged items.',
 "[Status: RESOLVED] Customer's issue with damaged items was reported to the warehouse for follow-up. The damaged item was to be adjusted in the customer's next delivery.",
 'typhoon')

In [48]:
x = "The customer inquired about the availability of a specific payment method. The customer also requested an update regarding the status of their order delivery."
y = "The agent indicated that the specific payment method is not currently available, but it may be offered in the future. The agent requested additional information to follow up on the delivery status, stating that the order is likely to be delivered on the scheduled delivery date. [Status: Unresolved]"

In [49]:
print(f"Customer problem: {x}\n\nAgent resolution: {y}")

Customer problem: The customer inquired about the availability of a specific payment method. The customer also requested an update regarding the status of their order delivery.

Agent resolution: The agent indicated that the specific payment method is not currently available, but it may be offered in the future. The agent requested additional information to follow up on the delivery status, stating that the order is likely to be delivered on the scheduled delivery date. [Status: Unresolved]


In [2]:
a = "Agent requested the reason for changing the delivery date and then processed the customer's request to reschedule the delivery. Agent informed the warehouse about the customer's new delivery preference. The delivery was rescheduled.\n\nCORE PROBLEM STATEMENT and RESOLUTION will be rewritten to follow the exact format required:\n\nCORE PROBLEM STATEMENT: Customer reported that items were not delivered to their store. Customer requested to change the delivery date. Customer was not available for delivery on the scheduled delivery date.\nRESOLUTION: Agent processed the customer's request to reschedule the delivery to a later date. Agent informed the warehouse about the customer's new delivery preference. RESOLVED\n\nLet's correct that to fit the required format:\n\nCORE PROBLEM STATEMENT:\nCustomer reported that items were not delivered to their store. Customer requested to change the delivery date. Customer was not available for delivery on the scheduled delivery date.\n\nRESOLUTION:\nAgent processed the customer's request to reschedule the delivery to a later date. Agent informed the warehouse about the customer's new delivery preference. RESOLVED"
print(a)

Agent requested the reason for changing the delivery date and then processed the customer's request to reschedule the delivery. Agent informed the warehouse about the customer's new delivery preference. The delivery was rescheduled.

CORE PROBLEM STATEMENT and RESOLUTION will be rewritten to follow the exact format required:

CORE PROBLEM STATEMENT: Customer reported that items were not delivered to their store. Customer requested to change the delivery date. Customer was not available for delivery on the scheduled delivery date.
RESOLUTION: Agent processed the customer's request to reschedule the delivery to a later date. Agent informed the warehouse about the customer's new delivery preference. RESOLVED

Let's correct that to fit the required format:

CORE PROBLEM STATEMENT:
Customer reported that items were not delivered to their store. Customer requested to change the delivery date. Customer was not available for delivery on the scheduled delivery date.

RESOLUTION:
Agent processed

In [32]:
prompt_template = """
## ROLE
Analyze customer support conversations for Tindahang Tapat to extract standardized problem-resolution pairs for RAG knowledge base.

## INPUT DATA
CONVERSATION: {conversation}
QUERY TYPE: {query_type}
SPECIFIC ISSUE: {specific_issue}

## OUTPUT REQUIREMENTS

### CORE PROBLEM STATEMENT (2-3 sentences)
- Main customer concern only (if multiple, pick primary)
- Based on explicit customer statements, not assumptions
- Use objective, factual language

### RESOLUTION (2-3 sentences or "Unresolved")
**RESOLVED**: Concrete action taken (refund processed, item dispatched, issue fixed, credits applied)
**Unresolved**: Promises, explanations, escalations, or reassurances only

### OPERATION TYPE
- Determine based on context clues in the conversation data:
  - **typhoon**: Weather-related disruptions, storm mentions, emergency protocols, "bagyo", "ulan", "baha", weather delays
  - **christmas**: Holiday season references, December timeframes, seasonal delays, "Pasko", Christmas rush
  - **normal**: Default for all other cases
- If unclear, default to "normal"

## NORMALIZATION RULES
**Replace ALL specifics with generic terms:**
- Names/stores/IDs → "customer", "customer's store", "customer order"
- ALL products → "items" (regardless of brand/type)
- Dates/times → "scheduled delivery date", "delivery time"
- Locations → "delivery location", "delivery area"
- Weather/traffic → "weather conditions", "traffic conditions"
- Tech issues → "mobile application became unresponsive", "payment processing failed", "authentication failed"

## PROHIBITED LANGUAGE
Remove all emotional/subjective terms: frustrated, angry, disappointed, apologetic, terrible, great, complained, pleased, etc.
Use neutral verbs: reported, stated, indicated, requested, received, experienced.

## OUTPUT FORMAT
CORE PROBLEM STATEMENT:
[normalized 2-3 sentences]

RESOLUTION:
[normalized 2-3 sentences with [Status: RESOLVED]] OR [Unresolved]

OPERATION TYPE:
[normal/typhoon/christmas]
"""

In [5]:
"CORE PROBLEM STATEMENT" not in str(None)

True

In [24]:
prompt_template = """
## ROLE
Analyze customer support conversations for Tindahang Tapat to extract standardized problem-resolution pairs for RAG knowledge base.

## INPUT DATA
CONVERSATION: {conversation}
QUERY TYPE: {query_type}
SPECIFIC ISSUE: {specific_issue}

## OUTPUT REQUIREMENTS

### CORE PROBLEM STATEMENT (2-3 sentences)
- Main customer concern only (if multiple, pick primary)
- Based on explicit customer statements, not assumptions
- Use objective, factual language

### RESOLUTION (2-3 sentences or "Unresolved")
**RESOLVED**: Concrete action taken (refund processed, item dispatched, issue fixed, credits applied)
**Unresolved**: Promises, explanations, escalations, or reassurances only

### OPERATION TYPE
- **typhoon**: Weather-related disruptions, storm mentions, emergency protocols
- **christmas**: Holiday season references, December timeframes, seasonal delays
- **normal**: Default for all other cases

## NORMALIZATION RULES
**Replace ALL specifics with generic terms:**
- Names/stores/IDs → "customer", "customer's store", "customer order"
- ALL products → "items" (regardless of brand/type)
- Dates/times → "scheduled delivery date", "delivery time"
- Locations → "delivery location", "delivery area"
- Weather/traffic → "weather conditions", "traffic conditions"
- Tech issues → "mobile application became unresponsive", "payment processing failed", "authentication failed"

## PROHIBITED LANGUAGE
Remove all emotional/subjective terms: frustrated, angry, disappointed, apologetic, terrible, great, complained, pleased, etc.
Use neutral verbs: reported, stated, indicated, requested, received, experienced.

## OUTPUT FORMAT
CORE PROBLEM STATEMENT:
[normalized 2-3 sentences]

RESOLUTION:
[normalized 2-3 sentences with [Status: RESOLVED]] OR [Unresolved]

OPERATION TYPE:
[normal/typhoon/christmas]
"""